<a href="https://colab.research.google.com/github/sedwardsmarsh/Marine-Mammal-Classifier/blob/master/Marine_Mammal_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Marine Mammal Classifier,** *in six steps!*

### Special thanks to:

*   Source for audio data: Watkins Marine Mammal Sound Database, Woods Hole Oceanographic Institution: https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm
*   Thanks to Todd Hayton for the python tutorial *Scraping by Example - Iterating through Select Items With Mechanize*: http://toddhayton.com/2015/01/09/scraping-by-example-ntu-edu/
*   This answer from stack exchange was helpful as well, thank you: https://stackoverflow.com/questions/5974595/download-all-the-linksrelated-documents-on-a-webpage-using-python
*   This post convinced me to use librosa. Thank you for the great resource: https://medium.com/@etown/great-results-on-audio-classification-with-fastai-library-ccaf906c5f52





# step 1. Configure Google Colab
### *Before running anything, you need to tell Colab that you want to use a GPU.* 
1.   You can do this by clicking on the ‘Runtime’ tab and selecting ‘Change runtime type’. 
2.   A pop-up window will open up with a drop-down menu. Select ‘GPU’ from the menu and click ‘Save’.









<img src="https://course.fast.ai/images/colab/03.png" alt="Click the 'Runtime' tab above and select 'Change runtime type'" height="100%" width="45%">
<img src="https://course.fast.ai/images/colab/04.png" alt="A pop-up window will open up with a drop-down menu. Select ‘GPU’ from the menu and click ‘Save’." height="50%" width="45%">

# step 2. Setup the environment

In [0]:
# connect to google drive
from google.colab import drive
from pathlib import Path
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
data_dir = root_dir + "Colab Notebooks/watkins_data/"
model_dir = root_dir + "Colab Notebooks/mmc_model/"
bash_path = "/content/gdrive/My\ Drive/Colab\ Notebooks/watkins_data/"
model_path = "/content/gdrive/My\ Drive/Colab\ Notebooks/mmc_model"

In [0]:
# create google drive directory to hold watkins marine mammal data
%mkdir {bash_path}
%mkdir {model_path}

In [0]:
# fetch the latest fast.ai version 
!curl -s https://course.fast.ai/setup/colab | bash

In [0]:
# install the latest versions of SoX and ffmpeg 
# !apt-get -qq install sox ffmpeg

# step 3. Download the data, *see Notice* 

### **Notice**: only run this script if you have not already done so, or if it was interrupted while running previously. ***Unfortunately, it takes a lot of time.***

##### script data source: https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm

In [0]:
# install the latest mechanize version
!pip -q install mechanize

In [0]:
#!/usr/bin/env python

# download all audio files from the 'Best Of' section on:
# https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm


import mechanize
import re
from time import sleep
import os
import pathlib

URL = 'https://whoicf2.whoi.edu/science/B/whalesounds/index.cfm'
DELAY = 0.5 

class WatkinsScraper:
    def __init__(self, url=URL, delay=DELAY):
        # initilize browser, url, delay and items array
        self.br = mechanize.Browser()
        self.url = url
        self.delay = delay
        self.items = []
        self.dl_links = []


    def scrape(self):
        '''
        Get the list of items in the first dropdown menu, "Common name", 
        submit the form for each item. 
        Using the response, save the files to this script's 
        directory.
        '''
        self.get_items()

        for item in self.items:
            # Skip invalid/blank item selections
            if "https" in str(item):
                self.get_links(str(item))

                label = ' '.join([label.text for label in item.get_labels()])

                # remove non alphanumeric characters
                label = re.sub('[^a-zA-Z]', '', label)


                # make directory
                if not os.path.exists(data_dir+"/"+label):
                    os.makedirs(data_dir+"/"+label)

                print("downloading %s..." % label)
                self.download_links(label)
                print("%s finished downloading!" % label)

        
    def get_items(self):
        '''
        Get the list of items in the first dropdown of the form.
        '''
        self.br.open(self.url)
        self.br.select_form('jump1')

        # get items from submit tag 
        self.items = self.br.form.find_control('getSpeciesCommon').get_items()


    def get_links(self, parent_url):
        '''
        Locates the links on a given webpage.
        '''
        temp_br = mechanize.Browser()
        temp_br.open(str(parent_url))

        # filetypes holds the extensions of the files we want to download.
        filetypes=[".wav"]
        # iterate through links inside browser on the page.
        for link in temp_br.links():
            # check if this link has the file extension we want.
            for ft in filetypes:
                if ft in str(link):
                    self.dl_links.append(link)


    def download_links(self, label):
        '''
        Downloads all links stored in a scraper obj. dl_links array.
        '''

        temp_br = mechanize.Browser()
        file_num = 0

        for link in self.dl_links:
            straight_url = re.sub('/science.*',link.url,link.base_url)
            sleep(self.delay)
            temp_br.retrieve(straight_url, 
                             data_dir+label+"/"+label+str(file_num)+".wav")
            file_num+=1

        # empty the links.
        self.dl_links = []


if __name__ == '__main__':
    scraper = WatkinsScraper()
    scraper.scrape()
    print("all sound files have finished downloading!")

# step 4. Convert all sound files to square spectrogram images, *see Notice*

### **Notice:** only run this script if you have not already done so, or if it was interrupted while running previously. **Unfortunately, it takes a lot of time.**

In [0]:
#!/usr/bin/env python

# convert all .wav files into square spectrogram images.

import os

def search():

    for folder in os.listdir(data_dir):
        # print("folder: %s" % data_dir+folder)
        for audio_file in os.listdir(data_dir+folder):
            # print("filename: %s" % filename)
            if audio_file.endswith('.wav'):
                temp_dir = data_dir+folder+"/"+audio_file
                # print(temp_dir)
                samples, sample_rate = librosa.load(temp_dir)
                fig = plt.figure(figsize=[0.72,0.72])
                ax = fig.add_subplot(111)
                ax.axes.get_xaxis().set_visible(False)
                ax.axes.get_yaxis().set_visible(False)
                ax.set_frame_on(False)
                #filename  = spectrogram_path/fold/Path(audio_file).name.replace('.wav','.png')
                filename = temp_dir.replace('.wav','.png')
                # plot mel-spectrogram
                S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
                plt.close('all')
        print("finished converting folder: %s" % data_dir+folder)
    print("finished converting all audio files to spectrograms!")


if __name__ == '__main__':
    # test_dir = os.path.join(data_dir+"")
    # print(data_dir)
    
    #filename  = spectrogram_path/fold/Path(audio_file).name.replace('.wav','.png')
    # os.system("sox {test_dir} -X 513 -y 513 -n spectrogram -hmr -o print.png -d 0:03")
    search()

# step 5. Make & view the dataset

In [0]:
from fastai.vision import *
batch_size = 64
data = ImageDataBunch.from_folder(path=data_dir, train=".", valid_pct=0.02, ds_tfms=[], size=244, bs=batch_size)
# data.split_by_rand_pct(valid_pct=0.2, seed=42)

In [0]:
# view a random selection of 9 elements in the dataset.
data.show_batch(rows=3, figsize=(10,10))

# step 6. Train the network

In [0]:
# create the learner and train the network
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

## **Notice:** If you previously trained the network and saved it, run the cell in "part a" and then skip to "part d"

###part a. 

In [0]:
# run this cell if the model was previously ran and saved, ignore otherwise.
learn.load(model_dir+"stage-1")

### part b. *actually* train the network

In [0]:
# train the network for 4 cycles
learn.fit_one_cycle(4)

In [0]:
# model is saved!
learn.save(model_dir+"stage-1")
learn.unfreeze()

### part c. tune the network

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(2, max_lr=slice(1e-3,1e-2))

### part d. view the network

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_top_losses(9, figsize=(18,18))

In [0]:
interp.plot_confusion_matrix(figsize=(10,10))

# Congratulations, you've trained a neural network to classify the sounds of Marine Mammals using fast.ai!

# *Thanks for checking out my project.* 
## *If you have any ideas on how to improve the data cleaning/mapping process, or if you know of larger marine mammal datasets, please let me know and open an issue on this repository!*